In [1]:
!pip install bm3d

In [2]:
!pip install python-multipart

In [3]:
!pip install FastAPI -q
!pip install uvicorn -q

In [4]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
async def root():
    return {"message": "Hello World"}

In [5]:
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart

In [6]:
!pip install io
from io import StringIO 

ERROR: Could not find a version that satisfies the requirement io
ERROR: No matching distribution found for io


In [7]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware

from fastapi.responses import Response
import os
from random import randint
import uuid
import io

from PIL import Image
import binascii
import io as StringIO

app = FastAPI()
db = []
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get('/myroot')
async def myroot():
    return {'hello': 'world2'}

@app.post("/images/")
async def create_upload_file(file: UploadFile = File(...)):

    file.filename = f"{uuid.uuid4()}.jpg"
    contents = await file.read()  # <-- Important!
    
    r_data = binascii.unhexlify(contents)
    stream = io.BytesIO(r_data)
    img = Image.open(stream)
    print(img.size)
    db.append(contents)

    return {"filename": file.filename}


@app.get("/images/")
async def read_random_file():

    # get a random file from the image db
    random_index = randint(0, len(db) - 1)

    # return a response object directly as FileResponse expects a file-like object
    # and StreamingResponse expects an iterator/generator
    response = Response(content=db[random_index])

    return response

In [8]:
 !pip install fastapi nest-asyncio pyngrok uvicorn python-multipart bm3d aiofiles

In [9]:
!pip install --upgrade ibm-cos-sdk

In [12]:
!pip install opencv-python

In [13]:
from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware

from fastapi.responses import Response
import os
from random import randint
import uuid

from PIL import Image
from io import BytesIO

from skimage import io, img_as_float
import bm3d
import cv2
import numpy as np
from PIL import ImageEnhance
from skimage.restoration import denoise_nl_means, estimate_sigma
from PIL import Image

import numpy as np

import base64

from fastapi.responses import FileResponse

import ibm_boto3
from ibm_botocore.client import Config,ClientError

from pydantic import BaseModel

######################## IBM COS
api_key = 'iHC4jacDY-Bn0_Gsaz4VV_-iPuMFeeGaNKHksHxKKRSC'
service_instance_id = 'crn:v1:bluemix:public:cloud-object-storage:global:a/26941411f7da4860b4c3e3ed96a54149:95ebcf90-1ba6-4918-b31f-d0cb60b03c3c::'
auth_endpoint = 'https://iam.bluemix.net/oidc/token'
service_endpoint = 'https://s3.jp-tok.cloud-object-storage.appdomain.cloud'

cos = ibm_boto3.resource('s3',
ibm_api_key_id=api_key,
ibm_service_instance_id=service_instance_id,
ibm_auth_endpoint=auth_endpoint,
config=Config(signature_version='oauth'),
endpoint_url=service_endpoint)
########################

######################## Item
class Item(BaseModel):
    name: str
########################

app = FastAPI()
db = []
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.get('/')
async def root():
    return {'hello': 'world'}

@app.get('/myroot')
async def myroot():
    return {'hello': 'world2'}

@app.post("/denoiseimage/")
async def deNoiseColor_Img(file: UploadFile = File(...)):


    file.filename = f"{uuid.uuid4()}.png"
    print(file.filename)
    contents = await file.read()  # <-- Important!

    stream = BytesIO(contents)

    i = Image.open(stream).convert("RGB")
    
    stream.close()
    
    image = np.asarray(i)
    
    image = image/255
    
    #denoise color image
    row,col,ch = image.shape
    mean = 0
    var = 0.1
    sigma = var**1
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    BM3D_denoised_image1 = bm3d.bm3d(image, .2, stage_arg=bm3d.BM3DStages.ALL_STAGES)
    print(type(BM3D_denoised_image1))
    im = Image.fromarray((BM3D_denoised_image1 * 255).astype(np.uint8))
    buffered = BytesIO()
    im.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue())
    return img_str

    
@app.post("/sharpimage/")
async def Sharpening_Img(file: UploadFile = File(...)):

        file.filename = f"{uuid.uuid4()}.png"
        print(file.filename)
        contents = await file.read()  # <-- Important!
        
        stream = BytesIO(contents)
        image = Image.open(stream).convert("RGB")
        stream.close()
        
        enhancer=ImageEnhance.Sharpness(image)
        f=.5
        fn=enhancer.enhance(f)
        
        buffered = BytesIO()
        fn.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue())
        return img_str
    
    
    
@app.post("/uploadimage/")
async def Upload_Img(file: UploadFile = File(...)):

        val = uuid.uuid4()
        file.filename = f"{uuid.uuid4()}.png"
        val = str(val)
        print(file.filename)
        contents = await file.read()  # <-- Important!
        
        stream = BytesIO(contents)
        i = Image.open(stream).convert("RGB")
#         image = np.asarray(i)
        stream.close()
        
        i.save(file.filename)
#         io.imsave(file.filename,image)
        try:
            print("Starting file transfer for {0} to bucket: {1}\n".format(file.filename, "my-test-bucket-0"))
            part_size = 1024 * 1024 * 5
            file_threshold = 1024 * 1024 * 15
            transfer_config = ibm_boto3.s3.transfer.TransferConfig(
                multipart_threshold=file_threshold,
                multipart_chunksize=part_size
            )
            with open(file.filename, "rb") as file_data:
                cos.Object("my-test-bucket-0",val).upload_fileobj(
                    Fileobj=file_data,
                    Config=transfer_config
                )
            print("Transfer for {0} Complete!\n".format(file.filename))
        except Exception as e:
            print("Unable to complete multi-part upload: {0}".format(e))
        
        
@app.post("/downloadimage/")      
async def get_image(item_name: Item):
    
    print(item_name.name)
    item_name = item_name.name
    bucket_name = "my-test-bucket-0"
    print("Retrieving item from bucket: {0}, key: {1}".format(bucket_name, item_name))
    try:
        file = cos.Object(bucket_name, item_name).get()
        d = file["Body"].read()
        stream = BytesIO(d)
        i = Image.open(stream).convert("RGB")
        stream.close()
        buffered = BytesIO()
        i.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue())
        return img_str
        
    except Exception as e:
        print("Unable to retrieve file contents: {0}".format(e))
        
        
@app.get("/getbucketimage/")       
async def get_bucket_images():
    bucket_name = "my-test-bucket-0"
    print("Retrieving bucket contents from: {0}".format(bucket_name))
    l = []
    try:
        files = cos.Bucket(bucket_name).objects.all()
        for file in files:
            print("Item: {0} ({1} bytes).".format(file.key, file.size))
            
            
            item_name = file.key
            bucket_name = "my-test-bucket-0"
            print("Retrieving item from bucket: {0}, key: {1}".format(bucket_name, item_name))
            try:
                file = cos.Object(bucket_name, item_name).get()
                d = file["Body"].read()
                stream = BytesIO(d)
                i = Image.open(stream).convert("RGB")
                stream.close()
                buffered = BytesIO()
                i.save(buffered, format="JPEG")
                img_str = base64.b64encode(buffered.getvalue())
                l.append(img_str)
            except Exception as e:
                print("Unable to retrieve file contents: {0}".format(e))
                
        return l
    except Exception as e:
        print("Unable to retrieve bucket contents: {0}".format(e))

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(7999)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=7999)

INFO:     Started server process [14976]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7999 (Press CTRL+C to quit)


Public URL: http://8b13481cccd7.ngrok.io
INFO:     37.111.128.215:0 - "GET / HTTP/1.1" 200 OK
INFO:     37.111.128.215:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     58.27.225.4:0 - "GET / HTTP/1.1" 200 OK
INFO:     37.111.128.215:0 - "GET /myroot HTTP/1.1" 200 OK
INFO:     37.111.128.215:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     182.176.130.6:0 - "GET /myroot HTTP/1.1" 200 OK
